In [1]:
import sys

print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


# 반복 가능한 객체 알아보기

- 시퀀스를 생성하는 객체.
- 한번 생성하고 다 사용하면 소진되는 객체.

# 1. 제녀레이터 표현식 알아보기

In [1]:
import collections.abc as cols

## 제너레이터 표현식으로 제너레이터 객체 생성하기

In [2]:
g = (x for x in range(5))

In [3]:
g

<generator object <genexpr> at 0x7fa4bc0ea050>

In [4]:
isinstance(g, cols.Generator)

True

### 제너레이터는 한번 다 사용하면 다시 사횽할 수 없다.

In [5]:
next(g);next(g);next(g);next(g);next(g);

In [10]:
next(g)


StopIteration: 

# 2.  이터레이터

`__iter__`, `__next__` 가 구현되어 있는 객체

## 파이썬은 함수와 속성을 동일하게 관리하므로 일단 이름만 있으면 동일한 것으로 인식

In [11]:
class A:
    __iter__ = True
    __next__ = True

In [12]:
issubclass(A, cols.Iterator)

True

## 내부에 로직을 넣기

#### `__iter__`는 자기 자신을 리턴

#### `__next__` 를 통해서 다음 값을 리턴

In [13]:
class Itt:
    def __init__(self, value):
        self.value = value
        self.i = 0
        
    def __iter__(self):
        # iterable객체의 경우 / __iter__에서 iterator객체를 리턴해야합니다.
        return self
    
    def __len__(self):
        return len(self.value)

    def __next__(self):
        if self.i < len(self):
            value = self.value[self.i]
            self.i += 1
            return value
        else:
            raise StopIteration

### 반복자 객체를 생성하고 소진한다.

- 한번 생성된 객체는 다 소진하면 다시 사용할 수 없다.

In [14]:
itt = Itt('a b c'.split())

In [15]:
next(itt), next(itt), next(itt)

('a', 'b', 'c')

In [16]:
next(itt)

StopIteration: 

## 반복처리하는 함수 만들어보기

- 재귀함수 호출로 반복적인 값을 처리할 수 있다.

In [17]:
def iterator(values):
    # iterator는 아님. 
    i = 0
    def _iterator():                  # 내부 함수를 정의해서 재귀함수로 사용
        nonlocal i
        try:
            value = values[i]         # 전달된 값 하나씩 반환 
        except IndexError:
            raise StopIteration
        i += 1
        return value
    return _iterator                  # 내부 함수 전달 

In [18]:
itt = iterator([1, 2, 3, 4, 5])

In [19]:
type(itt)

function

In [20]:
isinstance(itt, cols.Iterator)

False

In [21]:
itt(), itt(), itt(), itt(), itt(),

(1, 2, 3, 4, 5)

### 모든 원소를 다 처리하면 예외처리

In [22]:
itt()

StopIteration: 

# 3.  반복 가능한 객체 이해하기

## 시퀀스 객체 생성 후 반복가능한 객체로 변환

- 동적으로 반복할 객체로 변환이 가능
- 반복자 객체로 변환하는 것 


In [23]:
itb = 'abcde'

In [24]:
iter(itb)

In [25]:
iter([1,2,3,4])

In [26]:
iter((1,2,3,4))

## 새로운 반복형 클래스 정의

iterable객체는 -> iter함수를 통해서 -> iterator

### 이 클래스는 내부 조회할 수 있는 스페셜 메소드만 정의

In [27]:
class Getable:
    def __init__(self, value):
        self.value = value
    
    def __getitem__(self, name):
        return self.value[name]

In [28]:
gb = Getable([1, 2, 3, 4, 5])

In [29]:
gb[2], gb[1]

(3, 2)

### 반복형과 반복자 여부 확인 

In [30]:
isinstance(gb, cols.Iterable), isinstance(gb, cols.Iterator)

(False, False)

In [31]:
# iter함수를 통해서 iterator가 생성

iter(gb)

In [32]:
isinstance(iter(gb), cols.Iterator)

True

In [33]:
for n in gb:
    print(n, end=' ')

1 2 3 4 5 

# for 문 작동원리

In [34]:
# for문에서 반복시켜서 print해주는 로직 -> 함수

In [35]:
def for_print(obj):
    it = iter(obj) # iterator를 생성
    while True:
        try:
            print(next(it), end=' ')
        except StopIteration:
            break

In [36]:
for_print('abcde')

a b c d e 

In [37]:
it = iter('12345')

for_print(it)

1 2 3 4 5 

In [38]:
for_print(gb)

1 2 3 4 5 

## iter 함수는

1. `__iter__`메소드를 실행시킨다. 생성된 객체(iterator)를 리턴한다.
2. `__iter__`메소드가 없다면.. `__getitem__`을 찾아서.. 구현되어 있다면    
    0부터 넣어서 값을 찾습니다. 언제까지... StopIteration에러가 발생할때까지

In [39]:
class Itb:
    def __init__(self, value):
        self.value = value
        
    def __iter__(self):
        return Itt(self.value)



In [40]:
class Itt:
    def __init__(self, value):
        self.value = value
        self.i = 0
        
    def __iter__(self):
        # iterable객체의 경우 / __iter__에서 iterator객체를 리턴해야합니다.
        return self
    
    def __len__(self):
        return len(self.value)

    def __next__(self):
        if self.i < len(self):
            value = self.value[self.i]
            self.i += 1
            return value
        else:
            raise StopIteration

# Iterable + Iterator 

In [41]:
class ItbWithNext:
    def __init__(self, value):
        self.value = value
        self.i = 0

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i < len(self.value):
            value = self.value[self.i]
            self.i += 1
            return value
        else:
            raise StopIteration

In [42]:
itwn = ItbWithNext([1, 2, 3, 4, 5])

In [43]:
for i in itwn:
    print(i)

1
2
3
4
5


In [44]:
for_print(itwn)

In [45]:
for_print(itwn)

In [46]:
class ItbWithNext:
    def __init__(self, value):
        self.value = value
        self.i = 0

    def __iter__(self):
        self.i = 0
        return self
    
    def __next__(self):
        if self.i < len(self.value):
            value = self.value[self.i]
            self.i += 1
            return value
        else:
            raise StopIteration

In [47]:
itwn = ItbWithNext((1, 2, 3, 4, 5))

In [48]:
for_print(itwn)

1 2 3 4 5 

In [49]:
for_print(itwn)

1 2 3 4 5 

In [50]:
for i in itwn:
    print(i, end=', ')

1, 2, 3, 4, 5, 

## ItbWithNext객체를 통해서 조합가능한 순열 찾기


input

('빨', '주', '노')

ouput

(빨, 주) (빨 노) (주 빨) (주 노) (노 빨) (노 주)

In [51]:
colors = ItbWithNext(['빨', '주', '노'])

colors = ['빨', '주', '노']

for color1 in colors: # <- 객체의 위치정보가 정해짐
    for color2 in colors: # <- 여기서도 위치정보가 필요함
        if color1 != color2:
            print(color1, color2)

빨 주
빨 노
주 빨
주 노
노 빨
노 주
